In [1]:
from collections import Counter
import random

import numpy as np
import pyarrow.dataset as ds
import duckdb
import pandas as pd
import numpy as np
import seaborn as sns
import plotly as pl
import plotly.express as px
import json

from plotly.graph_objs import Figure

INDICADORES = {
    "AFD": "Adequação da Formação Docente",
    "ICG": "Complexidade de Gestão da Escola",
    "IED": "Esforço Docente",
    "ATU": "Média de Alunos por Turma",
    "HAD": "Média de Horas-aula diária",
    "DSU": "Percentual de Docentes com Curso Superior",
    "TDI": "Taxas de Distorção Idade-série"
}

DIMENSIONS = {
    "Ano": "NU_ANO_CENSO",
    "Nome da região geográfica": "NO_REGIAO",
    "Nome da Unidade da Federação": "NO_UF",
    "Nome da Mesorregião": "NO_MESORREGIAO",
    "Nome da Microrregião": "NO_MICRORREGIAO"
}

con = duckdb.connect('../file.db')

microdados = ds.dataset("../data/transformed/microdados.parquet", format="parquet", partitioning="hive")
con.register("microdados", microdados)

# indicadores = {}
# for indicador in INDICADORES:
#     indicadores[indicador] = ds.dataset(f"../data/transformed/indicadores/{indicador}.parquet", format="parquet", partitioning="hive")
#     con.register(indicador, indicadores[indicador])

In [5]:

niveis_ensino = [
    'Educação Básica', 'Educação Infantil - creche',
    'Educação Infantil - pré-escola', 'Educação Infantil',
    'Ensino Fundamental - anos iniciais',
    'Ensino Fundamental - anos finais',
    'Ensino Fundamental',
    # 'Ensino Médio', 'Educação Profissional',
    # 'Educação Profissional Técnica',
    # 'Educação de Jovens e Adultos (EJA) - Ensino Fundamental',
    # 'Educação de Jovens e Adultos (EJA) - Ensino Médio',
    # 'Educação de Jovens e Adultos (EJA)',
    # 'Educação Especial Inclusiva',
    # 'Educação Especial Exclusiva',
    'Educação Especial'
]

# Quantida de matriculas por nivel de ensino
quantidade_matriculas_nivel_ensino = con.execute("""
                select
                    NU_ANO_CENSO as 'Ano',
                    cast(sum(QT_MAT_BAS) as bigint) as 'Educação Básica',
                    cast(sum(QT_MAT_INF_CRE) as bigint) as 'Educação Infantil - creche',
                    cast(sum(QT_MAT_INF_PRE) as bigint) as 'Educação Infantil - pré-escola',
                    cast(sum(QT_MAT_INF) as bigint) as 'Educação Infantil',
                    cast(sum(QT_MAT_FUND_AI) as bigint) as 'Ensino Fundamental - anos iniciais',
                    cast(sum(QT_MAT_FUND_AF) as bigint) as 'Ensino Fundamental - anos finais',
                    cast(sum(QT_MAT_FUND) as bigint) as 'Ensino Fundamental',
                    cast(sum(QT_MAT_MED) as bigint) as 'Ensino Médio',
                    cast(sum(QT_MAT_PROF) as bigint) as 'Educação Profissional',
                    cast(sum(QT_MAT_PROF_TEC) as bigint) as 'Educação Profissional Técnica',
                    cast(sum(QT_MAT_EJA_FUND) as bigint) as 'Educação de Jovens e Adultos (EJA) - Ensino Fundamental',
                    cast(sum(QT_MAT_EJA_MED) as bigint) as 'Educação de Jovens e Adultos (EJA) - Ensino Médio',
                    cast(sum(QT_MAT_EJA) as bigint) as 'Educação de Jovens e Adultos (EJA)',
                    cast(sum(QT_MAT_ESP_CC) as bigint) as 'Educação Especial Inclusiva',
                    cast(sum(QT_MAT_ESP_CE) as bigint) as 'Educação Especial Exclusiva',
                    cast(sum(QT_MAT_ESP) as bigint) as 'Educação Especial'
                from microdados
                group by NU_ANO_CENSO
                order by 1
            """
                                                 ).df()
quantidade_matriculas_nivel_ensino = quantidade_matriculas_nivel_ensino.melt(
        id_vars='Ano',
        var_name="Nível de ensino",
        value_name="Quantidade de matrículas",
        value_vars=niveis_ensino
     )

px.line(
    quantidade_matriculas_nivel_ensino,
    x='Ano',
    y="Quantidade de matrículas",
    color="Nível de ensino",
    markers=True,
    title="Quantidade de matrículas por nível de ensino"
)

In [6]:
# quantidade de matriculas por dependencia
DIMENSOES_COMUNS = {
    "Dependência Administrativa": "TP_DEPENDENCIA",
    "Categoria de escola": "TP_CATEGORIA_ESCOLA_PRIVADA",
    "Localização": "TP_LOCALIZACAO",
    "Localização diferenciada da escola": "TP_LOCALIZACAO_DIFERENCIADA"
}

def get_plot_quantidade_matriculas(dimensao: str) -> (pl.graph_objs.Figure, pd.DataFrame):
    df = con.execute(f"""
                    select
                        NU_ANO_CENSO as 'Ano',
                        {DIMENSOES_COMUNS[dimensao]} as '{dimensao}',
                        count(*) as 'Quantidade de matrículas'
                    from microdados
                    group by NU_ANO_CENSO, {DIMENSOES_COMUNS[dimensao]}
                    order by 1, 2
                """
    ).df()
    fig = px.bar(
        df,
        x='Ano',
        y="Quantidade de matrículas",
        color=dimensao,
        title=f"Quantidade de matrículas por {dimensao.lower()}"
    )
    return fig, df

In [7]:
fig, df = get_plot_quantidade_matriculas("Dependência Administrativa")
fig

In [8]:
fig, df = get_plot_quantidade_matriculas("Categoria de escola")
fig

In [9]:
fig, df = get_plot_quantidade_matriculas("Localização")
fig

In [10]:
fig, df = get_plot_quantidade_matriculas("Localização diferenciada da escola")
fig

In [28]:
# Quantidade de matriculas por região geográfica
DIMENSOES_GEOGRAFICAS = {
    "Região geográfica": "NO_REGIAO",
    "Unidade da Federação": "NO_UF",
    "Mesorregião": "NO_MESORREGIAO",
    "Microrregião": "NO_MICRORREGIAO"
}

def get_plot_quantidade_matriculas_geografico(
        dimensao_geografica: str,
        nivel_ensino: str
) -> (pl.graph_objs.Figure, pd.DataFrame):
    df = con.execute(f"""
                    select
                        NU_ANO_CENSO as 'Ano',
                        {DIMENSOES_GEOGRAFICAS[dimensao_geografica]} as '{dimensao_geografica}',
                        cast(sum(QT_MAT_BAS) as bigint) as 'Educação Básica',
                        cast(sum(QT_MAT_INF_CRE) as bigint) as 'Educação Infantil - creche',
                        cast(sum(QT_MAT_INF_PRE) as bigint) as 'Educação Infantil - pré-escola',
                        cast(sum(QT_MAT_INF) as bigint) as 'Educação Infantil',
                        cast(sum(QT_MAT_FUND_AI) as bigint) as 'Ensino Fundamental - anos iniciais',
                        cast(sum(QT_MAT_FUND_AF) as bigint) as 'Ensino Fundamental - anos finais',
                        cast(sum(QT_MAT_FUND) as bigint) as 'Ensino Fundamental',
                        cast(sum(QT_MAT_MED) as bigint) as 'Ensino Médio',
                        cast(sum(QT_MAT_PROF) as bigint) as 'Educação Profissional',
                        cast(sum(QT_MAT_PROF_TEC) as bigint) as 'Educação Profissional Técnica',
                        cast(sum(QT_MAT_EJA_FUND) as bigint) as 'Educação de Jovens e Adultos (EJA) - Ensino Fundamental',
                        cast(sum(QT_MAT_EJA_MED) as bigint) as 'Educação de Jovens e Adultos (EJA) - Ensino Médio',
                        cast(sum(QT_MAT_EJA) as bigint) as 'Educação de Jovens e Adultos (EJA)',
                        cast(sum(QT_MAT_ESP_CC) as bigint) as 'Educação Especial Inclusiva',
                        cast(sum(QT_MAT_ESP_CE) as bigint) as 'Educação Especial Exclusiva',
                        cast(sum(QT_MAT_ESP) as bigint) as 'Educação Especial'
                    from microdados
                    group by NU_ANO_CENSO, {DIMENSOES_GEOGRAFICAS[dimensao_geografica]}
                    order by 1
                """
    ).df()

    fig = px.bar(
        df,
        x="Ano",
        y=nivel_ensino,
        color=dimensao_geografica,
        title=f"Quantidade de matrículas de {nivel_ensino} por {dimensao_geografica.lower()}",
        labels={nivel_ensino: "Quantidade de matrículas"}
    )

    return fig, df


In [29]:
fig, df = get_plot_quantidade_matriculas_geografico("Região geográfica", "Ensino Médio")
fig

In [51]:
def get_opcoes_geograficas(dimensao: str) -> list[str]:
    column = DIMENSOES_GEOGRAFICAS[dimensao]
    return list(np.sort(microdados.to_table(columns=[column]).to_pandas()[column].unique()))
get_opcoes_geograficas("Microrregião")

['Adamantina',
 'Afonso Cláudio',
 'Aglomeração Urbana de São Luís',
 'Agreste Potiguar',
 'Agreste de Itabaiana',
 'Agreste de Lagarto',
 'Aimorés',
 'Alagoana do Sertão do São Francisco',
 'Alagoinhas',
 'Alegre',
 'Alfenas',
 'Almeirim',
 'Almenara',
 'Alta Floresta',
 'Altamira',
 'Alto Araguaia',
 'Alto Capibaribe',
 'Alto Guaporé',
 'Alto Mearim e Grajaú',
 'Alto Médio Canindé',
 'Alto Médio Gurguéia',
 'Alto Pantanal',
 'Alto Paraguai',
 'Alto Parnaíba Piauiense',
 'Alto Solimões',
 'Alto Taquari',
 'Alto Teles Pires',
 "Alvorada D'Oeste",
 'Amapá',
 'Amparo',
 'Andradina',
 'Andrelândia',
 'Angicos',
 'Anicuns',
 'Anápolis',
 'Apucarana',
 'Aquidauana',
 'Aracaju',
 'Aragarças',
 'Araguaína',
 'Arapiraca',
 'Araranguá',
 'Araraquara',
 'Arari',
 'Araripina',
 'Araxá',
 'Araçatuba',
 'Araçuaí',
 'Arinos',
 'Aripuanã',
 'Ariquemes',
 'Assaí',
 'Assis',
 'Astorga',
 'Auriflama',
 'Avaré',
 'Bacia de São João',
 'Baixa Verde',
 'Baixada Maranhense',
 'Baixo Cotinguiba',
 'Baixo Cur

In [ ]:
microdados.to_table(columns=["NO_UF"]).to_pandas()["NO_UF"].unique()

In [35]:
microdados.

In [32]:
regiao_geografica = list(df["Região geográfica"].unique())

['Centro-Oeste', 'Norte', 'Nordeste', 'Sudeste', 'Sul']

In [149]:
#quantidade_matriculas_geo = quantidade_matriculas_geo[quantidade_matriculas_geo["Região geográfica"] == "Norte"]

In [150]:
px.bar(
    quantidade_matriculas_geo,
    x=dimension,
    y="Quantidade de matrículas",
    color="Nível de ensino",
)

In [108]:
with open("../geojs-100-mun.json") as f:
    geojson = json.load(f)

In [34]:
# Quantidade de escolas
dimension = "Ano"
quantidade_escolas = con.execute(f"""
                select
                    {DIMENSIONS[dimension]} as '{dimension}',
                    count(CO_ENTIDADE) as 'Quantidade de escolas'
                from microdados
                group by {DIMENSIONS[dimension]}
                order by 1
            """
                                 ).df()
px.bar(
        quantidade_escolas,
        x=dimension,
        y='Quantidade de escolas',
        title=f"QUANTIDADE DE ESCOLAS X {dimension.upper()}"
    )